In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np


# Verifica GPU
print("GPU disponible:", tf.config.list_physical_devices('GPU'))

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
#Cargar datos
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.io
plt.rcParams['figure.figsize'] = [8, 8]
plt.rcParams.update({'font.size': 18})

mat_contents = scipy.io.loadmat(os.path.join("/home/ivana/ml_projects/allFaces.mat"))
faces = mat_contents['faces']
m = int(mat_contents['m'])
n = int(mat_contents['n'])
nfaces = np.ndarray.flatten(mat_contents['nfaces'])


In [ ]:
# Create etiquetas para cada cara
labels = np.zeros(faces.shape[1])
for i in range(len(nfaces)):
  labels[np.sum(nfaces[:(i)]):np.sum(nfaces[:(i+1)])] = i+1

# Agregarlas etiquetas al arreglo faces
faces = np.concatenate((labels.reshape((1, faces.shape[1])), faces), axis=0)



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
import numpy as np

#Ordenar el areglo en vectores columna
faces_aux = faces.T
X = faces_aux[:,1:]
y = faces_aux[:,0]
# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Imprimir las formas de los conjuntos de entrenamiento y prueba
print("Forma de X_train:", X_train.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_test:", y_test.shape)


In [ ]:
import numpy as np

# Calcular la cantidad de imágenes de cada dígito en X_train y X_test
num_imagenes_digito_train = {int(digito): int(np.sum(y_train == digito)) for digito in np.unique(y_train)}
num_imagenes_digito_test = {int(digito): int(np.sum(y_test == digito)) for digito in np.unique(y_test)}

# Imprimir la cantidad de imágenes de cada dígito en X_train y X_test
print("Número de imágenes de cada dígito en X_train:")
print(dict(sorted(num_imagenes_digito_train.items())))

print("\nNúmero de imágenes de cada dígito en X_test:")
print(dict(sorted(num_imagenes_digito_test.items())))

In [ ]:
# Convertir y_train a un DataFrame de pandas
df_train = pd.DataFrame({'label': y_train})

# Crear una cuadrícula de gráficos con una gráfica para cada dígito
g = sns.FacetGrid(df_train, height=7, aspect=5)
g.map(sns.countplot, 'label', palette="icefire",order=sorted(df_train['label'].unique()))

plt.show()

# Obtener el recuento de valores únicos en y_train
print(df_train['label'].value_counts().sort_index())


In [ ]:
#Imagen ejemplo
img = X_train[9,:]
img = img.reshape((m,n)).T
plt.imshow(img,cmap='gray')
plt.title("Ejemplo")
plt.axis("off")
plt.show()

In [ ]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255


In [ ]:
# Reshape
X_train = X_train.reshape(-1,n,m,1)
X_test = X_test.reshape(-1,n,m,1)

#Tamaños
print("X_train shape: ",X_train.shape)
print("y_train shape: ",y_train.shape)
print("X_test shape: ",X_test.shape)
print("y_test shape: ",y_test.shape)


In [ ]:
#One hot encoding
from tensorflow.keras.utils import to_categorical

# Restar 1 a las etiquetas para que empiecen desde 0
y_train = y_train - 1
y_test = y_test - 1

# Aplicar one-hot encoding especificando num_classes
y_train = to_categorical(y_train, num_classes=38)
y_test = to_categorical(y_test, num_classes=38)

# Verificar las dimensiones
num_clases = y_test.shape[1]
print("Forma de y_train después de one-hot:", y_train.shape)
print("Forma de y_test después de one-hot:", y_test.shape)
print("Número de clases:", num_clases)

# Verificar que tenemos todas las clases
print("Número único de clases en y_train:", len(np.unique(np.argmax(y_train, axis=1))))
print("Número único de clases en y_test:", len(np.unique(np.argmax(y_test, axis=1))))

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

model = Sequential()
#
model.add(Conv2D(filters = 8, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (n,m,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
#
model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))
# fully connected
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(38, activation = "softmax"))

In [ ]:
# Define the optimizer
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
epochs = 30  # for better result increase the epochs
batch_size = 300

In [ ]:
# data augmentation
datagen = ImageDataGenerator(
        featurewise_center = False,  # set input mean to 0 over the dataset
        samplewise_center = False,  # set each sample mean to 0
        featurewise_std_normalization = False,  # divide inputs by std of the dataset
        samplewise_std_normalization = False,  # divide each input by its std
        zca_whitening = False,  # dimesion reduction
        rotation_range = 0,  # randomly rotate images in the range 5 degrees
        zoom_range = 0, # Randomly zoom image 10%
        width_shift_range = 0,  # randomly shift images horizontally 10%
        height_shift_range = 0,  # randomly shift images vertically 10%
        horizontal_flip = False,  # randomly flip images
        vertical_flip = False)  # randomly flip images

datagen.fit(X_train)

In [ ]:
# Fit the model
history = model.fit(datagen.flow(X_train,y_train,batch_size=batch_size), validation_data=(X_test,y_test), epochs=epochs)


# Final evaluation of the model
scores = model.evaluate(X_test,y_test, verbose=0)
print("Accuracy: %0.2f%%" % (scores[1]*100))

In [ ]:
# Plot the loss and accuracy curves for training and validation 
plt.plot(history.history['val_loss'], color='g', label="validation loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
print("\nNúmero de imágenes de cada dígito en X_test:")
print(num_imagenes_digito_test)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Configuración
num_classes = 38
class_names = [f"Persona {i+1:02d}" for i in range(num_classes)]

print("="*70)
print("ANÁLISIS DEL MODELO - YALE FACES (38 PERSONAS)")
print("="*70)
print(f"Modelo: {model.name}")
print(f"Input shape del modelo: {model.input_shape}")
print(f"Shape de X_test: {X_test.shape}")
print(f"Shape de y_test: {y_test.shape}")

# Hacer predicciones DIRECTAMENTE (sin preprocesamiento)
print("\nHaciendo predicciones...")
predictions = model.predict(X_test, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1) if len(y_test.shape) > 1 else y_test

# Accuracy general
accuracy = np.mean(predicted_classes == true_classes)
print(f"\n{'='*70}")
print(f"✅ ACCURACY GENERAL: {accuracy*100:.2f}%")
print(f"   Predicciones correctas: {np.sum(predicted_classes == true_classes)}/{len(y_test)}")
print(f"   Predicciones incorrectas: {np.sum(predicted_classes != true_classes)}/{len(y_test)}")
print(f"{'='*70}")

# ===== ACCURACY POR PERSONA =====
print("\n" + "="*70)
print("ACCURACY POR PERSONA")
print("="*70)

person_accuracy = []
for person_id in range(num_classes):
    mask = true_classes == person_id
    if np.sum(mask) > 0:
        person_acc = np.mean(predicted_classes[mask] == true_classes[mask])
        person_accuracy.append((person_id, person_acc, np.sum(mask)))

# Ordenar por accuracy
person_accuracy.sort(key=lambda x: x[1], reverse=True)

print("\n🏆 TOP 10 - Mejor reconocimiento:")
for i, (person_id, acc, count) in enumerate(person_accuracy[:10], 1):
    print(f"  {i:2d}. {class_names[person_id]}: {acc*100:5.1f}% ({count} imágenes)")

print("\n⚠️  BOTTOM 10 - Necesitan mejorar:")
for i, (person_id, acc, count) in enumerate(person_accuracy[-10:], 1):
    print(f"  {i:2d}. {class_names[person_id]}: {acc*100:5.1f}% ({count} imágenes)")

In [ ]:
# Encontrar errores
errors = predicted_classes != true_classes
error_indices = np.where(errors)[0]

if len(error_indices) > 0:
    from collections import Counter
    
    print("\n" + "="*70)
    print("ANÁLISIS DE ERRORES")
    print("="*70)
    print(f"\nTotal de errores: {len(error_indices)} de {len(X_test)} ({len(error_indices)/len(X_test)*100:.2f}%)")
    
    # Confusiones más frecuentes
    confusions = [(true_classes[i], predicted_classes[i]) for i in error_indices]
    most_common = Counter(confusions).most_common(10)
    
    print("\n10 confusiones más frecuentes:")
    for i, ((true_id, pred_id), count) in enumerate(most_common, 1):
        print(f"  {i:2d}. {class_names[true_id]} → {class_names[pred_id]} : {count} veces")
    
    # Visualizar ejemplos de errores
    print("\nVisualizando ejemplos de errores...")
    n_errors_show = min(10, len(error_indices))
    
    fig, axes = plt.subplots(2, 5, figsize=(18, 8))
    axes = axes.ravel()
    
    for i, idx in enumerate(error_indices[:n_errors_show]):
        image = X_test[idx]
        true_id = true_classes[idx]
        pred_id = predicted_classes[idx]
        confidence = predictions[idx][pred_id]
        
        if len(image.shape) == 3 and image.shape[2] == 1:
            axes[i].imshow(image.squeeze(), cmap='gray')
        else:
            axes[i].imshow(image, cmap='gray')
        
        title = f"Real: {class_names[true_id]}\nPred: {class_names[pred_id]}\nConf: {confidence*100:.0f}%"
        axes[i].set_title(title, color='red', fontsize=9, fontweight='bold')
        axes[i].axis('off')
    
    plt.suptitle('Ejemplos de Errores de Clasificación', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("\n🎉 ¡Modelo perfecto! No hay errores en el conjunto de prueba.")

In [ ]:
# confusion matrix
import seaborn as sns
# Predict the values from the validation dataset
Y_pred = model.predict(X_test)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
f,ax = plt.subplots(figsize=(30, 30))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Blues",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Asumiendo que ya tienes tu generador configurado y entrenaste el modelo
# Por ejemplo: después de model.fit(...)

# 1. Crear y guardar class_names (si usas nombres generados)
class_names = [f"persona_{i+1}" for i in range(38)]  # O usa train_generator.class_indices si viene de carpetas
np.save('nombres_clases.npy', class_names)
print("Nombres de clases guardados como 'nombres_clases.npy'")

# 2. Guardar el modelo
model.save('modelo_rostros.h5')
print("Modelo guardado como 'modelo_rostros.h5'")

print("¡Todo guardado exitosamente!")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
# Cargar modelo
model = load_model('modelo_rostros.h5')

# Cargar nombres
class_names = np.load('nombres_clases.npy', allow_pickle=True).tolist()

print("Modelo y nombres cargados.")
# Ahora úsalos en predicciones: nombre = class_names[clase_predicha]

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# === REMAPEAR ETIQUETAS A 0,1,2,...,37 ===
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Convierte cualquier etiqueta a 0–37

# Verificar que ahora sí están en 0–37
print("Etiquetas originales únicas:", np.unique(y))
print("Etiquetas codificadas únicas (0 a 37):", np.unique(y_encoded))
print(f"Número de clases: {len(np.unique(y_encoded))}")  # Debe ser 38

# === Dividir con etiquetas codificadas ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Convertir a int
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# === GUARDAR EL MAPEO PARA USARLO DESPUÉS ===
np.save('label_encoder_classes.npy', label_encoder.classes_)  # IDs originales
class_names = [f"persona_{i+1}" for i in range(38)]
np.save('nombres_clases.npy', class_names)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# ================= CARGAR MODELO Y CLASES =================
MODEL_PATH = 'modelo_rostros.h5'
CLASES_PATH = 'nombres_clases.npy'

model = load_model(MODEL_PATH)
class_names = np.load(CLASES_PATH, allow_pickle=True).tolist()

print(f"Modelo y clases cargados. Total clases: {len(class_names)}")

# ================= TUS DATOS (ya los tienes) =================
# X_train, X_test, y_train, y_test → ya definidos
# Suponiendo que las imágenes están aplanadas: (n_samples, n_pixels)

IMG_HEIGHT = n # Cambia si usaste otro
IMG_WIDTH = m
N_PIXELS = IMG_HEIGHT * IMG_WIDTH  # Debe coincidir con X_train.shape[1]


# ================= FUNCIÓN: Predecir un batch de imágenes =================
def predecir_batch(X_batch, model, class_names):
    # Redimensionar: (n_samples, n_pixels) → (n_samples, 64, 64, 1)
    X_batch = X_batch.reshape(-1, IMG_HEIGHT, IMG_WIDTH, 1)
    X_batch = X_batch / 255.0  # Normalización

    # Predicción
    predicciones = model.predict(X_batch, verbose=0)
    clases_predichas = np.argmax(predicciones, axis=1)
    confianzas = np.max(predicciones, axis=1)

    return clases_predichas, confianzas, predicciones

# ================= EJEMPLO: Predecir en X_test =================
print("\nHaciendo predicciones en X_test...")

y_pred, confianzas, probs = predecir_batch(X_test, model, class_names)

# ================= MOSTRAR ALGUNOS RESULTADOS =================
print("\nPrimeras 10 predicciones en X_test:")
for i in range(min(10, len(X_test))):
    real = int(y_test[i])
    pred = y_pred[i]
    confianza = confianzas[i]
    nombre_real = class_names[real]
    nombre_pred = class_names[pred]
    acierto = "Correcto" if real == pred else "Incorrecto"
    
    print(f"Imagen {i}: Real: {nombre_real} | Pred: {nombre_pred} | Conf: {confianza*100:5.1f}% → {acierto}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# ================= CONFIGURACIÓN =================
IMG_HEIGHT = m   # Ajusta si usaste otro
IMG_WIDTH = n

# ================= FUNCIÓN CORREGIDA =================
def mostrar_imagen_con_prediccion(idx, X_test, y_test, y_pred, confianzas, class_names):
    # Redimensionar correctamente: (n_pixels,) → (height, width)
    img = X_test[idx].reshape(IMG_HEIGHT, IMG_WIDTH).T
    
    real = class_names[int(y_test[idx])]
    pred = class_names[y_pred[idx]]
    confianza = confianzas[idx] * 100
    
    plt.figure(figsize=(4, 5))
    plt.imshow(img, cmap='gray')
    color = 'green' if real == pred else 'red'
    plt.title(f'Real: {real}\nPred: {pred} ({confianza:.1f}%)', color=color, fontsize=12)
    plt.axis('off')
    plt.show()

# ================= MOSTRAR VARIAS IMÁGENES DE PRUEBA =================
print("Mostrando 5 ejemplos del conjunto de prueba (X_test):")

for i in [0, 10, 20, 30, 66]:  # Puedes cambiar los índices
    if i < len(X_test):
        print(f"\n--- Imagen {i} de X_test ---")
        mostrar_imagen_con_prediccion(i, X_test, y_test, y_pred, confianzas, class_names)
    else:
        print(f"Índice {i} fuera de rango (X_test tiene {len(X_test)} imágenes)")

In [ ]:
print("Mostrando solo errores en X_test:")
errores = np.where(y_test != y_pred)[0]

for i in errores[:9]:  # Primeros 5 errores
    print(f"\n--- ERROR en índice {i} ---")
    mostrar_imagen_con_prediccion(i, X_test, y_test, y_pred, confianzas, class_names)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy en X_test: {accuracy*100:.2f}%")




In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy en test: {accuracy*100:.2f}%")

# Reporte detallado
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred, target_names=class_names))